# Fetching Data using Spotipy

We will be using the Spotipy library to get the data from Spotify Web API. 3 dataset will be generated - 

* Meta-Data: Comprising of information about the public playlists generated by Spotify such as number of followers, desciption, tracks in the playlist etc.
.
* Songs Data: Information on the songs in each of the Playlists. This includes information about the artist, audio features of the songs, popularity etc.

* Artists Data: Information about artist for each of the songs such as artist popularity, artist genre, number of followers etc.

In [1]:
#Importing Libraries
import time
import pandas as pd
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials

Please use your own client_id and client_secret

In [2]:
#Initializing Spotipy
client_credentials_manager = SpotifyClientCredentials(client_id='3fe9f6b3dcc349fe85e3433aff2e6c64', client_secret='59259626c5264bc3b2692ee58ea60811')
sp = sp.Spotify(client_credentials_manager=client_credentials_manager)
username = 'spotify'

## Get meta-data

In [3]:
#Lists of Playlists for user 'spotify'
playlists = sp.user_playlists(username)
allPlaylistsId = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        allPlaylistsId.append(playlist['id'])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [4]:
#Information about the playlist
allPlaylistsData = []
playlistInfo = {}
for id in allPlaylistsId:
    try:
        results = sp.user_playlist(username, playlist_id=id)
        playlistInfo['Name'] = results['name']
        playlistInfo['Followers'] = results['followers']['total']
        playlistInfo['Description'] = results['description']
        playlistInfo['Tracks'] = len(results['tracks']['items'])
        playlistInfo['id'] = results['id']
        playlistInfo['href'] = results['href']
        playlistInfo['URL'] = results['external_urls']['spotify']
        playlistInfo['Collaborative'] = results['collaborative']
        allPlaylistsData.append(playlistInfo)
        playlistInfo = {}
    except:
        time.sleep(5)
        results = sp.user_playlist(username, playlist_id=id)
        playlistInfo['Name'] = results['name']
        playlistInfo['Followers'] = results['followers']['total']
        playlistInfo['Description'] = results['description']
        playlistInfo['Tracks'] = len(results['tracks']['items'])
        playlistInfo['id'] = results['id']
        playlistInfo['href'] = results['href']
        playlistInfo['URL'] = results['external_urls']['spotify']
        playlistInfo['Collaborative'] = results['collaborative']
        allPlaylistsData.append(playlistInfo)
        playlistInfo = {}
        continue

In [5]:
# Generate the csv dataset for above
allPlaylistsDf = pd.DataFrame(allPlaylistsData)
allPlaylistsDf.to_csv('playlists.csv')

## Get songs data

In [6]:
# Playlists and their Songs
trackPlaylist = {}
allTracksPlaylists = []
for id in allPlaylistsId:
            try:
                results = sp.user_playlist(username, playlist_id=id)
                for tracks in results['tracks']['items']:
                    if tracks['track'] == None:
                        continue
                    else:
                        trackPlaylist['Track'] = tracks['track']['id']
                        trackPlaylist['Playlist'] = id
                        allTracksPlaylists.append(trackPlaylist)
                        trackPlaylist = {}
            except:
                time.sleep(3)
                results = sp.user_playlist(username, playlist_id=id)
                for tracks in results['tracks']['items']:
                    trackPlaylist['Track'] = tracks['track']['id']
                    trackPlaylist['Playlist'] = id
                    allTracksPlaylists.append(trackPlaylist)
                    trackPlaylist = {}
                continue

In [7]:
# Generate the csv dataset for above
allTracksPlaylistsDf = pd.DataFrame(allTracksPlaylists)
allTracksPlaylistsDf.to_csv('TracksPlaylist.csv')

In [9]:
# Getting all the tracks in all playlists
allTracks = []
for k in allTracksPlaylists:
    allTracks.append(k['Track'])
allTracksClean = list(filter(lambda a: a != None, allTracks))

#Information about the songs
allTracksInfo = []
j = {}
offset = 0
remainingTracks = len(allTracksClean)
while offset < len(allTracksClean):
    try:
        if remainingTracks > 50:
            internalCounter = 0
            resultTracks = sp.tracks(allTracksClean[offset:offset+50])['tracks']
            resultFeatures = sp.audio_features(allTracksClean[offset:offset+50])
            while internalCounter < len(resultTracks):
                if resultFeatures[internalCounter] != None:
                    j = resultFeatures[internalCounter]
                j['name'] = resultTracks[internalCounter]['name']
                j['artist_name'] = resultTracks[internalCounter]['artists'][0]['name']
                j['artist_id'] = resultTracks[internalCounter]['artists'][0]['id']
                j['popularity'] = resultTracks[internalCounter]['popularity']
                j['available_markets'] = len(resultTracks[internalCounter]['available_markets'])
                j['duration_ms'] = resultTracks[internalCounter]['duration_ms']
                j['explicit'] = resultTracks[internalCounter]['explicit']
                internalCounter = internalCounter + 1
                allTracksInfo.append(j)   
                j={}
            offset = offset + 50
            remainingTracks = remainingTracks - 50
        else:
            internalCounter = 0
            resultTracks = sp.tracks(allTracksClean[offset:offset+remainingTracks])['tracks']
            resultFeatures = sp.audio_features(allTracksClean[offset:offset+remainingTracks])
            while internalCounter < len(resultTracks):
                if resultFeatures[internalCounter] != None:
                    j = resultFeatures[internalCounter]
                j['name'] = resultTracks[internalCounter]['name']
                j['artist_name'] = resultTracks[internalCounter]['artists'][0]['name']
                j['artist_id'] = resultTracks[internalCounter]['artists'][0]['id']
                j['popularity'] = resultTracks[internalCounter]['popularity']
                j['available_markets'] = len(resultTracks[internalCounter]['available_markets'])
                j['duration_ms'] = resultTracks[internalCounter]['duration_ms']
                j['explicit'] = resultTracks[internalCounter]['explicit']
                internalCounter = internalCounter + 1
                allTracksInfo.append(j)
                j={}
            offset = offset + remainingTracks
            remainingTracks = remainingTracks - remainingTracks
    except:
            time.sleep(5)
            if remainingTracks > 50:
                internalCounter = 0
                resultTracks = sp.tracks(allTracksClean[offset:offset+50])['tracks']
                resultFeatures = sp.audio_features(allTracksClean[offset:offset+50])
                while internalCounter < len(resultTracks):
                    if resultFeatures[internalCounter] != None:
                        j = resultFeatures[internalCounter]
                    j['name'] = resultTracks[internalCounter]['name']
                    j['artist_name'] = resultTracks[internalCounter]['artists'][0]['name']
                    j['artist_id'] = resultTracks[internalCounter]['artists'][0]['id']
                    j['popularity'] = resultTracks[internalCounter]['popularity']
                    j['available_markets'] = len(resultTracks[internalCounter]['available_markets'])
                    j['duration_ms'] = resultTracks[internalCounter]['duration_ms']
                    j['explicit'] = resultTracks[internalCounter]['explicit']
                    internalCounter = internalCounter + 1
                    allTracksInfo.append(j)
                    j={}
                offset = offset + 50
                remainingTracks = remainingTracks - 50
            else:
                internalCounter = 0
                resultTracks = sp.tracks(allTracksClean[offset:offset+remainingTracks])['tracks']
                resultFeatures = sp.audio_features(allTracksClean[offset:offset+remainingTracks])
                while internalCounter < len(resultTracks):
                    if resultFeatures[internalCounter] != None:
                        j = resultFeatures[internalCounter]
                    j['name'] = resultTracks[internalCounter]['name']
                    j['artist_name'] = resultTracks[internalCounter]['artists'][0]['name']
                    j['artist_id'] = resultTracks[internalCounter]['artists'][0]['id']
                    j['popularity'] = resultTracks[internalCounter]['popularity']
                    j['available_markets'] = len(resultTracks[internalCounter]['available_markets'])
                    j['duration_ms'] = resultTracks[internalCounter]['duration_ms']
                    j['explicit'] = resultTracks[internalCounter]['explicit']
                    internalCounter = internalCounter + 1
                    allTracksInfo.append(j)
                    j={}
                offset = offset + remainingTracks
                remainingTracks = remainingTracks - remainingTracks
                continue

In [10]:
# Generate the csv dataset for above
allTracksInfoDf = pd.DataFrame(allTracksInfo)
allTracksInfoDf.to_csv('allTracksInfo.csv')

## Get artists data

In [ ]:
allArtistsInfo = []
k = {}
offset = 0
remainingArtists = len(allTracksInfoDf['artist_id'])
while offset < len(allTracksInfoDf['artist_id']):
    try:
        if remainingArtists > 50:
            internalCounter = 0
            resultArtists = sp.artists(allTracksInfoDf['artist_id'][offset:offset+50])['artists']
            while internalCounter < len(resultArtists):
                k['artist_id'] = resultArtists[internalCounter]['id']
                k['artist_name'] = resultArtists[internalCounter]['name']
                k['artist_genres'] = resultArtists[internalCounter]['genres']
                k['popularity'] = resultArtists[internalCounter]['popularity']
                k['followers'] = resultArtists[internalCounter]['followers']['total']
                internalCounter = internalCounter + 1
                allArtistsInfo.append(k)
                print(len(allArtistsInfo))
                k={}
            offset = offset + 50
            remainingArtists = remainingArtists - 50
        else:
            internalCounter = 0
            resultArtists = sp.artists(allTracksInfoDf['artist_id'][offset:offset+remainingArtists])['artists']
            while internalCounter < len(resultArtists):
                k['artist_id'] = resultArtists[internalCounter]['id']
                k['artist_name'] = resultArtists[internalCounter]['name']
                k['artist_genres'] = resultArtists[internalCounter]['genres']
                k['popularity'] = resultArtists[internalCounter]['popularity']
                k['followers'] = resultArtists[internalCounter]['followers']['total']
                internalCounter = internalCounter + 1
                allArtistsInfo.append(k)
                print(len(allArtistsInfo))
                k={}
            offset = offset + remainingArtists
            remainingArtists = remainingArtists - remainingArtists
    except:
        time.sleep(5)
        if remainingArtists > 50:
            internalCounter = 0
            resultArtists = sp.artists(allTracksInfoDf['artist_id'][offset:offset+50])['artists']
            while internalCounter < len(resultArtists):
                k['artist_id'] = resultArtists[internalCounter]['id']
                k['artist_name'] = resultArtists[internalCounter]['name']
                k['artist_genres'] = resultArtists[internalCounter]['genres']
                k['popularity'] = resultArtists[internalCounter]['popularity']
                k['followers'] = resultArtists[internalCounter]['followers']['total']
                internalCounter = internalCounter + 1
                allArtistsInfo.append(k)
                print(len(allArtistsInfo))
                k={}
            offset = offset + 50
            remainingArtists = remainingArtists - 50
        else:
            internalCounter = 0
            resultArtists = sp.artists(allTracksInfoDf['artist_id'][offset:offset+remainingArtists])['artists']
            while internalCounter < len(resultArtists):
                k['artist_id'] = resultArtists[internalCounter]['id']
                k['artist_name'] = resultArtists[internalCounter]['name']
                k['artist_genres'] = resultArtists[internalCounter]['genres']
                k['popularity'] = resultArtists[internalCounter]['popularity']
                k['followers'] = resultArtists[internalCounter]['followers']['total']
                internalCounter = internalCounter + 1
                allArtistsInfo.append(k)
                print(len(allArtistsInfo))
                k={}
            offset = offset + remainingArtists
            remainingArtists = remainingArtists - remainingArtists
            continue

In [89]:
allArtistsInfoDf = pd.DataFrame(allArtistsInfo)
allArtistsInfoDf.to_csv('allArtistsInfoDf.csv')